### Deep learning inference

- Detect tree objects from dataset

In [ ]:
from urbantree.setting import Setting

#SETTING = "settings/by_dop80c_1312/deepforest_r1/setting.yaml"
#SETTING = "settings/by_dop80c_1312/deepforest_r2/setting.yaml"
#SETTING = "settings/by_dop80c_1312/deepforest_r3/setting.yaml"
#SETTING = "settings/opendata_luftbild_dop60_1312/deepforest_r1/setting.yaml"
#SETTING = "settings/opendata_luftbild_dop60_1312/deepforest_r2/setting.yaml"
SETTING = "settings/opendata_luftbild_dop60_1312/deepforest_r3/setting.yaml"

setting = Setting.load_deepforest_setting(SETTING)
setting['model_inference_config']

- detect tree objects from dataset and the resulting bounding boxes DataFrame are saved in pickle objects.
- The object detection can be performed on different patch sizes and
  no extra nms is run on aggregated results.

In [ ]:
from urbantree.deepforest.detection import infer_images

infer_images(**setting)

- Render tree canopy raster images according to the resulting bounding boxes DataFrame saved in pickle objects.

In [ ]:
from urbantree.deepforest.detection import postprocess_render_images

dataset_img_pattern="*.tiff"

postprocess_render_images(**setting,
                          dataset_img_pattern=dataset_img_pattern)

- calculate difference among two datasets

In [ ]:
from urbantree.deepforest.detection import calc_diff

# tune inference parameters to increase outcome
diff_from_inference_param = {
  'confident_min_bbox_size': 20,
  'confident_min_score': 0.9,
  'morphology_factor': 1,
  'patch': [
    {'patch_size': 1200,
    'patch_overlap': 0.3,
    'iou_threshold': 0.8,
    'score_thresh': 0.95},
    {'patch_size': 800,
    'patch_overlap': 0.3,
    'iou_threshold': 0.8,
    'score_thresh': 0.9},
    {'patch_size': 200,
    'patch_overlap': 0.2,
    'iou_threshold': 0.7,
    'score_thresh': 0.8},
    {'patch_size': 96,
    'patch_overlap': 0.18,
    'iou_threshold': 0.6,
    'score_thresh': 0.8}]}
diff_to_inference_param = {
  'confident_min_bbox_size': 30,
  'confident_min_score': 0.8,
  'morphology_factor': 3,
  'patch': [
    {'patch_size': 1200,
    'patch_overlap': 0.3,
    'iou_threshold': 0.8,
    'score_thresh': 0.2},
    {'patch_size': 800,
    'patch_overlap': 0.3,
    'iou_threshold': 0.8,
    'score_thresh': 0.1},
    {'patch_size': 200,
    'patch_overlap': 0.2,
    'iou_threshold': 0.7,
    'score_thresh': 0.01},
    {'patch_size': 96,
    'patch_overlap': 0.18,
    'iou_threshold': 0.6,
    'score_thresh': 0.01}]}

calc_diff(diff_from_setting_path="settings/opendata_luftbild_dop60_1312/deepforest_r3/setting.yaml",
          diff_to_setting_path="settings/by_dop80c_1312/deepforest_r2/setting.yaml",
          aggregate_iou_threshold=0.4,
          diff_iou_threshold=0.08,
          diff_cover_threshold=0.1,
          diff_from_inference_param=diff_from_inference_param,
          diff_to_inference_param=diff_to_inference_param,
          output_bbox_dir='temp/diff/b',
          output_debug_img_dir='temp/diff/debug',
          concurrency=11) 

- generate bbox output for mapping

In [2]:
from urbantree.deepforest.detection import create_bbox_shapefile

create_bbox_shapefile(src_img_dir='aerial_images_resampled/opendata_luftbild_dop60_1312',
                    src_bbox_diff='temp/diff/b/diff',
                    output_shp_path='temp/diff/diff.shp',
                    size_min_threshold=200, 
                    iou_threshold=0.2)

create_bbox_shapefile(src_img_dir='aerial_images_resampled/opendata_luftbild_dop60_1312',
                    src_bbox_diff='interim/opendata_luftbild_dop60_1312/deepforest_r3/inference/b',
                    output_shp_path='temp/diff/2017.shp',
                    size_min_threshold=200,
                    iou_threshold=0.3)

create_bbox_shapefile(src_img_dir='aerial_images_resampled/opendata_luftbild_dop60_1312',
                    src_bbox_diff='interim/by_dop80c_1312/deepforest_r2/inference/b',
                    output_shp_path='temp/diff/2020.shp',
                    size_min_threshold=200,
                    iou_threshold=0.3)


100%|██████████| 1474/1474 [00:56<00:00, 25.95it/s]


total: 20502


100%|██████████| 1492/1492 [15:14<00:00,  1.63it/s]


total: 935209


100%|██████████| 1484/1484 [13:24<00:00,  1.84it/s]


total: 842348
